In [1]:
import functools
import os
import gzip
import pandas as pd

import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [6]:
events_df = pd.read_csv(gzip.open(os.path.join('data', 'events.csv.gz')),
                 delimiter='|', parse_dates=['start_time'])
events_df.head()

/home/inonpe/jupyter/dtu_bda_project/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,26,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,source,start_time,end_time,latitude,longitude,postal_code,all_day,free,featured,...,venue_escaped,address_escaped,description_escaped,categories_escaped,tags_escaped,recurrent_escaped,performers_escaped,price_escaped,url_escaped,city_escaped
0,E0-001-094943257-4,eventful,2016-08-19 19:00:00,NaN,40.700800,-73.713300,NaN,0.0,NaN,NaN,...,NuSpecies,NaN,"In this workshop, Beverley Leon, one of NuSpe...",{u'category': [{u'name': u'Health &amp; Wellne...,NaN,NaN,NaN,NaN,http://newyorkcity.eventful.com/events/digesti...,Elmont
1,E0-001-094206304-7,eventful,2016-08-27 21:00:00,NaN,40.721321,-73.988212,10002,0.0,NaN,NaN,...,Arlene's Grocery,NaN,<br>,{u'category': [{u'name': u'Other &amp; Miscell...,NaN,NaN,NaN,$10.00,http://newyorkcity.eventful.com/events/disconn...,New York
2,E0-001-092628830-9,eventful,2016-07-15 20:00:00,NaN,40.719288,-73.961708,11211,0.0,NaN,NaN,...,Music Hall of Williamsburg,NaN,NaN,{u'category': [{u'name': u'Concerts &amp; Tour...,NaN,NaN,"{u'performer': {u'name': u'Emily King', u'crea...",25.00 - 25.00,http://newyorkcity.eventful.com/events/emily-k...,Brooklyn
3,E0-001-093348618-5,eventful,2016-06-28 18:00:00,NaN,40.668600,-74.114700,NaN,0.0,NaN,NaN,...,Dorothy Harrington Council Chambers,NaN,NaN,"{u'category': [{u'name': u'Neighborhood', u'id...",NaN,NaN,NaN,NaN,http://newyorkcity.eventful.com/events/cancell...,Bayonne
4,E0-001-095381670-0,eventful,2016-08-08 19:00:00,NaN,40.742716,-73.992934,NaN,0.0,NaN,NaN,...,3 Sheets Saloon,NaN,Join us for the 4th year of helping kids who ...,{u'category': [{u'name': u'Fundraising &amp; C...,NaN,NaN,NaN,NaN,http://newyorkcity.eventful.com/events/operati...,New York


In [23]:
venues_df = pd.read_csv('../js/data/venue_info.csv', index_col='venue')[['lat', 'lon']]
venues_df

,lat,lon
venue,,
Madison Square Garden,40.750667,-73.993460
MoMA Museum,40.762415,-73.977439
Beacon Theatre,40.780514,-73.981085
Bowery Ballroom,40.720600,-73.993900
Carnegie Hall,40.765126,-73.979924
Javits Center,40.757756,-74.002686
Lincoln Center,40.772464,-73.983489
Playstation Theater,40.757877,-73.986233
Radio City Music Hall,40.759976,-73.979977


In [34]:
def events_in_radius(venue, precision):
    venue_lat = venues_df.loc[venue].lat
    venue_lon = venues_df.loc[venue].lon
    return events_df\
        [['start_time', 'latitude', 'longitude', 'title_escaped']]\
        [lambda df: 
         (df.latitude <= venue_lat + precision) & 
         (df.latitude >= venue_lat - precision) &
         (df.longitude <= venue_lon + precision) & 
         (df.longitude >= venue_lon - precision)
        ].assign(venue=venue)
        
events_in_radius('Carnegie Hall', 0.003).shape

(519, 5)

In [41]:
def join_events_and_venue(precision):
    return pd.concat(events_in_radius(venue, precision)
                     for venue in venues_df.index)

join_events_and_venue(0.003).to_csv('./for_d3/events_venues.csv', index=False)